In [ ]:
from jax import config
config.update('jax_enable_x64', True)

import jax
import gpjax as gpx
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt

from uncprop.core.inverse_problem import Posterior
from uncprop.models.vsem.inverse_problem import generate_vsem_inv_prob_rep
from uncprop.models.vsem.surrogate import fit_vsem_surrogate

from uncprop.core.surrogate import construct_design
from uncprop.utils.gpjax_models import construct_gp, train_gp_hyperpars
from uncprop.utils.plot import set_plot_theme
from uncprop.utils.grid import Grid

colors = set_plot_theme()
key = jr.key(53212)

In [ ]:
key, key_inv_prob, key_surrogate = jr.split(key, 3)

# settings
noise_sd = 0.1
n_months = 24

inverse_problem_settings = {
    'key': key_inv_prob,
    'par_names': ['kext', 'av'],
    'n_windows': n_months, # note: windows are months
    'n_days_per_window': 30,
    'observed_variable': 'lai',
    'noise_cov_tril': noise_sd * jnp.identity(n_months)
}

surrogate_settings = {
    'key': key_surrogate,
    'n_design': 15,
    'design_method': 'lhc'
}


# construct inverse problem
posterior = generate_vsem_inv_prob_rep(**inverse_problem_settings)

# surrogate-based inverse problem
surrogate_posterior = fit_vsem_surrogate(posterior=posterior, **surrogate_settings)

In [ ]:
grid = Grid(low=posterior.support[0],
            high=posterior.support[1],
            n_points_per_dim=[50, 50],
            dim_names=posterior.prior.par_names)

In [ ]:
fig, ax = grid.plot(f=posterior._get_log_density_function(),
                    title='posterior')

In [ ]:
from uncprop.utils.grid import DensityComparisonGrid

density_grid = DensityComparisonGrid(grid=grid, distributions={'exact': posterior})

In [ ]:
fig, ax = density_grid.plot('exact', normalized=True, log_scale=False)

In [ ]:
# sampling posterior
key, key_mcmc = jr.split(key, 2)

samp = posterior.sample(key, n=3000, num_warmup_steps=500) # returns HMCState with samp.position (n, 1, 2)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(samples[:,0])
plt.show()

### Fit Surrogate

In [ ]:
key, key_design = jr.split(key, 2)

# sample design points
design = construct_design(key=key_design,
                          design_method='lhc', 
                          n_design=15, 
                          prior=posterior.prior,
                          f=lambda x: posterior.log_density(x))

# fit log-posterior surrogate
surrogate = VSEMSurrogate(design=design, exact_posterior=posterior)
surrogate.summarize_fit()

In [ ]:
# summarize surrogate mean and sd
pred = surrogate(grid.flat_grid)

grid.plot(z=pred.mean, title='surrogate mean')
grid.plot(z=jnp.sqrt(pred.variance), title='surrogate sd')

In [ ]:
from uncprop.core.surrogate import LogDensGPSurrogate

surrogate_post = LogDensGPSurrogate(log_dens=surrogate, support=posterior.support)

In [ ]:
dists = {
    'exact': posterior,
    'mean': surrogate_post.expected_surrogate_approx(),
    'eup': surrogate_post.expected_density_approx()
}

density_comparison = DensityComparisonGrid(grid=grid, distributions=dists)

In [ ]:
density_comparison.plot(['exact', 'mean', 'eup'],
                         normalized=True, log_scale=False,
                         max_cols=3)